<a href="https://colab.research.google.com/github/christophermalone/stat360/blob/main/Handout10_Jackknife.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Handout #10 : The "Leave-One-Out" Concept, i.e. Jackknife

## Load Tidyverse

The tidyverse R package will be used to assist with reading in the dataset into the current R session.

In [ ]:
#@title Load Tidyverse
#load tidyverse package
library(tidyverse)

Warning message in system("timedatectl", intern = TRUE):
“running command 'timedatectl' had status 1”
── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.5     ✔ purrr   0.3.4
✔ tibble  3.1.5     ✔ dplyr   1.0.7
✔ tidyr   1.1.4     ✔ stringr 1.4.0
✔ readr   2.0.2     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()





---




## Simple Case: "Leave-One-Out" for a Mean

Let's suppose we have a simple data vector, say $\bf{y}$.

$$\bf{y} = \begin{bmatrix}
2 \\
3 \\
5 \\
8 \\
10 \\
\end{bmatrix}$$


Putting this vector into R.

In [67]:
y <- c(2, 3, 5, 8, 10)

Next, getting the mean of $\bf{y}$ using the mean() function.

In [68]:
mean(y)

[1] 5.6

The minus argument can be used to *temporarily* withhold an observation from calculations done on an object.  For example, the following will calculate the mean of y without the $1^{st}$ observation.  


In [69]:
#What does y look like without the 1st observation
cat("y vector without 1st observation: ")
y[-1]

cat("\n")

#Getting the mean of y without the 1st observation
cat("Mean of y without 1st observation:")
mean( y[-1] )

y vector without 1st observation: 

[1]  3  5  8 10


Mean of y without 1st observation:

[1] 6.5

Next, continue by removing the $2^{nd}$ observation from consideration.

In [70]:
#What does y look like without the 2nd observation
cat("y vector without 2nd observation: ")
y[-2]

cat("\n")

#Getting the mean of y without the 1st observation
cat("Mean of y without 2nd observation:")
mean( y[-2] )

y vector without 2nd observation: 

[1]  2  5  8 10


Mean of y without 2nd observation:

[1] 6.25

Continue with remaining observations.

In [71]:
#Getting the mean of y without the 3rd observation
cat("Mean of y without 3rd observation:")
mean( y[-3] )

cat("\n")

#Getting the mean of y without the 4th observation
cat("Mean of y without 4th observation:")
mean( y[-4] )

cat("\n")

#Getting the mean of y without the 5th observation
cat("Mean of y without 5th observation:")
mean( y[-5] )

Mean of y without 3rd observation:

[1] 5.75


Mean of y without 4th observation:

[1] 5


Mean of y without 5th observation:

[1] 4.5

## Leave-One-Out Algorithm for a Mean

Step 1: Create an output vector to hold the outcomes from the algorithm

In [ ]:
output = rep(0,5)
output

[1] 0 0 0 0 0

Step #2:  Use a for() loop to cycle through for each of the five observations


In [ ]:
for(i in 1:5){
               output[i] = mean( y[-i] )
             }

cat("Output vector after for() loop: ")
output

Output vector after for() loop: 

[1] 6.50 6.25 5.75 5.00 4.50

### Writing your own function for the "Leave-One-Out" Method

Consider the following code that will create a function in R.  A function is simply a collection of code.  A well-written function allows your code to be more flexible and have increased functionality.

A function in R should:

*   Have a good name
*   Have clearly defined chunks of code with comments
*   Return something
*   Often functions contain input arguments



In [ ]:
#A custom function which has name mean.jackknife
#This function takes one input argument, z
#This function returns a vector of the jackknife mean

mean.jackknife = function(z){
    
    #Find the length of z
    n = length(z)
    
    #Setup output vector
    output = rep(0,n)
    
    #Loop for iterations
    for(i in 1:n){
        output[i] = mean( z[-i])
    }
    
    #Return the output vector
    return(output)

}

Next, let's use our custom function to obtain the jackknife mean of the vector $\bf{y}$.

In [ ]:
mean.jackknife(y)

[1] 6.50 6.25 5.75 5.00 4.50

## "Leave-One-Out" for Regression Coefficients

Next, let us consider the application of "leave-one-out" to coefficients from a simple linear regression model.

Suppose we have two vectors, say $\bf{y} = [2, 3, 5, 8, 10]$ and $\bf{x} = [1, 2, 3, 4, 5]$.   Next, put these vectors in a data.frame named $\bf{mydata}$.

$$\bf{mydata} = \begin{bmatrix}
          2 & 1 \\
          3 & 2\\
          5 & 3\\
          8 & 4\\
          10 & 5\\
          \end{bmatrix}
$$ 


In [ ]:
#Getting the two vectors in R
y_vector = c(2, 3, 5, 8, 10)
x_vector = c(1, 2, 3, 4, 5)

#Creating the mydata data.frame
mydata <- data.frame(y=y_vector, x = x_vector)
mydata

y,x
<dbl>,<dbl>
2,1
3,2
5,3
8,4
10,5


The lm() function will be used to fit the least-square regression model for $y \sim x$. 

$$
E(Y|X=x) = \beta_{0} + \beta_{1}*x
$$

In [ ]:
myfit <- lm(y~x, data = mydata)
myfit


Call:
lm(formula = y ~ x, data = mydata)

Coefficients:
(Intercept)            x  
       -0.7          2.1  


The coefficients from the linear model can be obtained by specifying $coefficients on the linear model object.

In [ ]:
myfit$coefficients

(Intercept)           x 
       -0.7         2.1

The following code can be used to obtain the second coefficient from the linear model object, i.e. $\hat{\beta}_{1}$

In [ ]:
myfit$coefficients[2]

x 
2.1

### Writing your own DFBETA Function 

The following is a custom function, named betahat.jackknife(), that is used to obtain the jackknife estimates from a simple linear model.

In [ ]:
#A custom function for obtaining DFBETAs
#INPUTS
#      This function takes two input arguments
#      1) A simple linear regression object, and 
#      2) data.frame used for fitting SLR model
#OUTPUTS
#     This function returns a data.frame of the jackknife 
#     estimates of the regression coefficients

betahat.jackknife=function(slr_object,data_for_fit){
    
    #Getting the number of rows in data_for_fit
    n = dim(data_for_fit)[1]
    
    #Creating the output data frame, column 1 for beta0hat and 
    # column 2 for beta1hat
    output = data.frame( beta0hat=numeric() , beta1hat=numeric() )

    #Looping through data, Beta0hat will be put into column 1
    # and Beta1hat will be put into column 2 
    for(i in 1:n){
        fit_i=lm(formula(slr_object),data=data_for_fit[-i,])
        output[i,1]=fit_i$coefficients[1]
        output[i,2]=fit_i$coefficients[2]
    }
    
    #Return the output vector
    return(output)
    
}

Next, let's use our custom function to obtain the jackknife estimates of the coefficients from the simple linear regression model. 

In [ ]:
betahat.jackknife(myfit, data_for_fit = mydata)

,beta0hat,beta1hat
,<dbl>,<dbl>
1,-1.9000000,2.400000
2,-0.3428571,2.028571
3,-0.5500000,2.100000
4,-0.6571429,2.057143
5,-0.5000000,2.000000


The DFBeta terms can be computed as follows:

*   $\mbox{DFBeta}_{0} = \hat{\beta}_{0} - \hat{\beta}_{0 (-i)} $
*  $\mbox{DFBeta}_{1} = \hat{\beta}_{1} - \hat{\beta}_{1 (-i)} $



In [ ]:
#Putting the jackknife estimates into a data.frame called outcomes
outcomes <- betahat.jackknife(myfit, data_for_fit = mydata)

#Computing the DFBeta terms for each parameter estimate
DFBETAs <- (
            outcomes
            %>% mutate(
                        DFBeta0 = (-0.7 - beta0hat), 
                        DFBeta1 = (-2.1 - beta1hat), 
                      )
          )

#Printing the resulting data.frame
cat("DFBETA for y-intercept & slope")
DFBETAs

DFBETA for y-intercept & slope

,beta0hat,beta1hat,DFBeta0,DFBeta1
,<dbl>,<dbl>,<dbl>,<dbl>
1,-1.9000000,2.400000,1.20000000,-4.500000
2,-0.3428571,2.028571,-0.35714286,-4.128571
3,-0.5500000,2.100000,-0.15000000,-4.200000
4,-0.6571429,2.057143,-0.04285714,-4.157143
5,-0.5000000,2.000000,-0.20000000,-4.100000


## "Leave-One-Out" for Prediction

Again, suppose we have two vectors, say $\bf{y} = [2, 3, 5, 8, 10]$ and $\bf{x} = [1, 2, 3, 4, 5]$.   

$$\bf{mydata} = \begin{bmatrix}
          2 & 1 \\
          3 & 2\\
          5 & 3\\
          8 & 4\\
          10 & 5\\
          \end{bmatrix}
$$ 

In [ ]:
#Getting the two vectors in R
y_vector = c(2, 3, 5, 8, 10)
x_vector = c(1, 2, 3, 4, 5)

#Creating the mydata data.frame
mydata <- data.frame(y=y_vector, x = x_vector)
mydata

y,x
<dbl>,<dbl>
2,1
3,2
5,3
8,4
10,5


The lm() function will be used to fit the least-square regression model for $y \sim x$. The summary() and anova() functions will be used to obtain additonal output from this modle.

In [ ]:
myfit <- lm(y~x, data = mydata)
summary(myfit)


Call:
lm(formula = y ~ x, data = mydata)

Residuals:
   1    2    3    4    5 
 0.6 -0.5 -0.6  0.3  0.2 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)   
(Intercept)  -0.7000     0.6351  -1.102  0.35086   
x             2.1000     0.1915  10.967  0.00162 **
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.6055 on 3 degrees of freedom
Multiple R-squared:  0.9757,	Adjusted R-squared:  0.9676 
F-statistic: 120.3 on 1 and 3 DF,  p-value: 0.001623


In [ ]:
anova(myfit)

,Df,Sum Sq,Mean Sq,F value,Pr(>F)
,<int>,<dbl>,<dbl>,<dbl>,<dbl>
x,1,44.1,44.1000000,120.2727,0.001623197
Residuals,3,1.1,0.3666667,NA,NA


The residual vector can be computed as 

$$
\bf{Residuals} = ( \bf{y} - \hat{E}{(Y|x)} )
$$



$$\bf{Residuals} = \begin{bmatrix}
          2 \\
          3\\
          5\\
          8\\
          10\\
          \end{bmatrix} - \begin{bmatrix}
          1.4 \\
          3.5\\
          5.6\\
          7.7\\
          9.8\\
          \end{bmatrix} = \begin{bmatrix}
          +0.6 \\
          -0.5\\
          -0.6\\
          +0.3\\
          +0.2\\
          \end{bmatrix}
$$ 

Getting the RMSE via brute force for the orginal observations.

In [ ]:
#Using the predict() function to get predicted value for each observation in the mydata data.frame
predictedy = predict(myfit, newdata = mydata)

#Next, computing the RMSE via brute force
sqrt(sum((y-predictedy)^2)/3)

[1] 0.6055301

## "Leave-One-Out" Algorithm for Prediction Accuracy

First, let us consider the process for leaving out the 1st observation.

In [ ]:
#First, obtain a fit without the first observation
myfit_minus_1=lm(y~x,data=mydata[-1,])

#Next, obtain the prediction for the first observation
# Here, the model was fit without this observation being included
predictedy_1=predict(myfit_minus_1,newdata=mydata[1,])

#Finally, obtain the squared residual value from this fit
(y[1]-predictedy_1)^2



1 
2.25

Now, let us extend the process completed above to all observations.

Step 1: Create an output vector to hold the outcomes from the algorithm

In [ ]:
output = rep(0,5)
output

[1] 0 0 0 0 0

Step #2:  Use a for() loop to cycle through for each of the five observations

In [ ]:
for(i in 1:5){
              myfit_minus_i = lm( y~x, data = mydata[-i,] )
              predictedy_i = predict( myfit_minus_i, newdata = mydata[i,] )
              output[i] = ( y[i] - predictedy_i )^2
             }

cat("Output vector after for() loop: ")
output

Output vector after for() loop: 

[1] 2.2500000 0.5102041 0.5625000 0.1836735 0.2500000

Getting the Mean Square Error (MSE) and Root Mean Square Error (RMSE) estimates using the jackknife approach.

In [ ]:
#Get the Mean Squared Error
cat("MSE estimate via Jackknife is: ")
mean(output)

cat("\n")
#Get the Root of the Mean Squared Error
cat("RMSE estimate via Jackknife is: ")
sqrt(mean(output))

MSE estimate via Jackknife is: 

[1] 0.7512755


RMSE estimate via Jackknife is: 

[1] 0.8667615

### Writing your own RMSE Jackknife function for Prediction Accuracy

The following custom function can be used to obtain a jackknife estimate for the Root Mean Square Error.

In [ ]:
#A custom function for obtaining RMSE via Jackknife
#INPUTS
#      This function takes two input arguments
#      1) A simple linear regression object, and 
#      2) data.frame used for fitting SLR model
#OUTPUTS
#     This function returns as output
#      1) vector of squared residuals
#      2) the jackknife estimate of RMSE

rmse.jackknife=function(slr_object,data_for_fit){
    
    #Getting the number of rows in data
    n = dim(data_for_fit)[1]

    #Keeping a copy of orginial y (used in computed residual)
    originaly = slr_object$model[,1]
    
    #Creating the output vector to save squared residuals
    output = rep(0,n)

    #Looping through data 
    for(i in 1:n){
        fit_minus_i = lm( formula(slr_object), data=data_for_fit[-i,] )
        predictedy_i = predict( fit_minus_i , newdata=data_for_fit[i,] )
        output[i] = ( originaly[i] - predictedy_i )^2
    }
    
    #Return the output vector and jackknife rmse
    list(SquaredResids=output,Jackknife_RMSE=sqrt(mean(output)))
}

Using the rmse.jackknife() function for the data considered here.

In [ ]:
rmse.jackknife(myfit, mydata)

$SquaredResids
[1] 2.2500000 0.5102041 0.5625000 0.1836735 0.2500000

$Jackknife_RMSE
[1] 0.8667615

Finally, the jackknife RMSE (0.867) is somewhat higher than the RMSE from the model fit (0.61).  This is expected because the jackknife RMSE is using a *more true error term* as the observation being predicted is witheld while fitting the model for the jackknife approach.

End of Document